In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split 

In [2]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

In [3]:
#!pip install torch

In [4]:
df = pd.read_csv('tweets_blue_prep.csv')

/Users/juanpablomereaotermin/.pyenv/versions/3.8.6/envs/MooDy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10,11,12,13,14,18,19,20,21,23,24,29,30,31,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df.head()

,Unnamed: 0,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,...,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,fecha,bid,ask
0,0,NaN,2020-04-27,NaN,NaN,http://m.uy/uc751169,0,1254570018089635840,NaN,NaN,...,Montevideo,Javier Carrizo,JavierCarrizo2,125197.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
1,1,NaN,2020-04-27,NaN,NaN,NaN,0,1254567307373879296,NaN,NaN,...,"Buenos Aires, Argentina",cris,usacrist_cris,504.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
2,2,NaN,2020-04-27,NaN,NaN,https://twitter.com/orchala66/status/125456588...,16,1254567736199516166,NaN,NaN,...,URUGUAY🇺🇾,Elizabeth Izzi 🦋🇺🇾,ElizabethIzzi,57034.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
3,3,NaN,2020-04-27,NaN,NaN,https://twitter.com/ESPNUruguay/status/1254490...,1,1254563921450672130,NaN,NaN,...,"Montevideo, Uruguay",Santiago,Santi_Bas_00,19463.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
4,4,NaN,2020-04-27,NaN,NaN,NaN,0,1254571152829222912,NaN,NaN,...,Punta del Este,Libertad,AnaRuth29310478,2554.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00


In [6]:
fd = df[['created_at',
    'urls',
    'hashtags',
    'user_favourites_count',
    'retweet_count',
    'user_followers_count',
    'user_screen_name',
    'user_name',
    'user_verified',
    'text',
    'bid',
    'ask']]

In [7]:
ai = fd[['created_at','bid','ask']]
ai.sample(15)

,created_at,bid,ask
386072,2020-10-06,NaN,NaN
307396,2020-09-06,NaN,NaN
503407,2020-10-26,$ 184.40,$ 190.00
4750,2020-04-25,NaN,NaN
428193,2020-10-08,$ 152.45,$ 158.00
1035957,2021-04-19,$ 138.80,$ 143.00
121952,2020-06-22,$ 118.55,$ 126.90
1256055,2021-06-27,NaN,NaN
503579,2020-10-26,$ 184.40,$ 190.00
876102,2021-03-13,NaN,NaN


In [8]:
X_train,X_test = train_test_split(fd, test_size=0.90, random_state=42)

In [9]:
X_train

,created_at,urls,hashtags,user_favourites_count,retweet_count,user_followers_count,user_screen_name,user_name,user_verified,text,bid,ask
128330,2020-06-24,NaN,DonáPlasma,43006.0,0.0,2565.0,lucioaberastain,Lucio Pedro Aberastain Ponte,False,unidad ciudadana máximo kirchner cel apoyo d o...,$ 119.65,$ 127.95
444301,2020-10-11,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,nose siendo fumadora todavía morí covidy sigo ...,NaN,NaN
702157,2020-12-21,https://twitter.com/DiegoMTroffe/status/134083...,NaN,40944.0,3.0,1244.0,agusscascella,AGUSTINAAAA,False,nuevo año nuevo dijo covid,$ 144.20,$ 150.00
920387,2021-03-29,NaN,Covid_19,8918.0,0.0,3746.0,egodoyvallejos,Esteban,True,casos confirmados covid argentina cosa complic...,$ 137.80,$ 142.00
1045223,2021-04-21,https://twitter.com/realmadrid/status/13848866...,NaN,30628.0,0.0,1951.0,DiegoAlbornoz8,Diego Albornoz,False,federico valverde dado positivo test covid pro...,$ 139.80,$ 144.00
...,...,...,...,...,...,...,...,...,...,...,...,...
110268,2020-06-17,https://www.panasenutah.com/2020/06/17/utah-do...,17jun,1153.0,0.0,8215.0,jacmibelrosas,jacmibelrosas,False,jun utah do nuevas muertes covid nuevos casos,$ 115.75,$ 124.05
259178,2020-08-22,NaN,NaN,11973.0,40.0,14715.0,danielafurst,Daniela Furst,True,gente elijo juntarme miedo covid sino mala onda,NaN,NaN
131932,2020-06-25,NaN,NaN,772.0,2.0,3705.0,UNDEFArgentina,UNDEF,True,ii jornada actualidad aeronáutica impacto covi...,$ 119.65,$ 127.95
671155,2020-12-05,NaN,NaN,1193.0,0.0,127.0,__AbigailR___,Noee✿,False,llegue diciembre notificado policia haber covi...,NaN,NaN


In [10]:
X_train.dtypes

created_at                object
urls                      object
hashtags                  object
user_favourites_count    float64
retweet_count            float64
user_followers_count     float64
user_screen_name          object
user_name                 object
user_verified             object
text                      object
bid                       object
ask                       object
dtype: object

In [11]:
a = X_train.iloc[89,-3]
b = X_train.text
a

'discurso kirchnerista culpa presidente culpó junto cambio medios manifestantes covid quejó herencia recibida omitiendo últimos veinte años vice parte gobierno dieciséis'

In [12]:
# Inicialización
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 16
NCLASSES = 2

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [13]:
import transformers 

In [45]:
text = a

In [75]:
PRE_TRAINED_MODEL_NAME = 'BETO-cased'
tokenizer = transformers.BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME , do_lower_case =True)

HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/api/models/BETO-cased

In [74]:
tokens = tokenizer.basic_tokenizer.tokenize((txt))
token_ids = tokenizer.basic_tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

AttributeError: 'BasicTokenizer' object has no attribute 'convert_tokens_to_ids'

In [69]:
encoding = tokenizer.encode_plus(
    txt,
    max_length = 10,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    pad_to_max_length = True,
    return_attention_mask = True,
    return_tensors = 'pt'
) 

In [70]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [71]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', 'disc', '##urs', '##o', 'ki', '##rch', '##ner', '##ista', 'cu', '[SEP]']
tensor([  101,  5860,  9236,  2080, 11382, 11140,  3678, 11921, 12731,   102])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [77]:
#!pip install pysentimiento

In [78]:
from pysentimiento import SentimentAnalyzer

analyzer = SentimentAnalyzer(lang="es")

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/841 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

/Users/juanpablomereaotermin/.pyenv/versions/3.8.6/envs/MooDy/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [98]:
predict = analyzer.predict(X_train.text)

TypeError: 'float' object is not iterable

In [97]:
print(predict)

SentimentOutput(output=NEG, probas={NEG: 0.570, NEU: 0.427, POS: 0.003})


In [87]:
df.head()

,Unnamed: 0,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,...,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified,fecha,bid,ask
0,0,NaN,2020-04-27,NaN,NaN,http://m.uy/uc751169,0,1254570018089635840,NaN,NaN,...,Montevideo,Javier Carrizo,JavierCarrizo2,125197.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
1,1,NaN,2020-04-27,NaN,NaN,NaN,0,1254567307373879296,NaN,NaN,...,"Buenos Aires, Argentina",cris,usacrist_cris,504.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
2,2,NaN,2020-04-27,NaN,NaN,https://twitter.com/orchala66/status/125456588...,16,1254567736199516166,NaN,NaN,...,URUGUAY🇺🇾,Elizabeth Izzi 🦋🇺🇾,ElizabethIzzi,57034.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
3,3,NaN,2020-04-27,NaN,NaN,https://twitter.com/ESPNUruguay/status/1254490...,1,1254563921450672130,NaN,NaN,...,"Montevideo, Uruguay",Santiago,Santi_Bas_00,19463.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00
4,4,NaN,2020-04-27,NaN,NaN,NaN,0,1254571152829222912,NaN,NaN,...,Punta del Este,Libertad,AnaRuth29310478,2554.0,NaN,NaN,False,2020-04-27,$ 109.00,$ 118.00


In [ ]:
import money

In [ ]:
import party 

In [ ]:
import peron

In [83]:
txt

'discurso kirchnerista culpa presidente culpó junto cambio medios manifestantes covid quejó herencia recibida omitiendo últimos veinte años vice parte gobierno dieciséis'

In [107]:
txt_2 = 'que feliz que estoy con la gestion de alberto fernandez viva pero'
txt_3 = 'hoy hubieron 200 contagios de covid, que porqueria!'

In [108]:
analyzer.predict(txt_2), analyzer.predict(txt_3)

(SentimentOutput(output=POS, probas={POS: 0.998, NEU: 0.001, NEG: 0.001}),
 SentimentOutput(output=NEG, probas={NEG: 0.999, POS: 0.000, NEU: 0.000}))

In [109]:
analyzer.predict(txt_2), analyzer.predict(txt_3)

(SentimentOutput(output=POS, probas={POS: 0.998, NEU: 0.001, NEG: 0.001}),
 SentimentOutput(output=NEG, probas={NEG: 0.999, POS: 0.000, NEU: 0.000}))

In [110]:
emotion_analyzer.predict(txt_2), emotion_analyzer.predict(txt_3)

(EmotionOutput(output=joy, probas={joy: 0.992, surprise: 0.002, fear: 0.002, others: 0.002, disgust: 0.001, sadness: 0.001, anger: 0.001}),
 EmotionOutput(output=anger, probas={anger: 0.822, disgust: 0.165, surprise: 0.004, fear: 0.003, sadness: 0.002, joy: 0.002, others: 0.002}))

In [100]:
from pysentimiento import EmotionAnalyzer

emotion_analyzer = EmotionAnalyzer(lang="es")

https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /Users/juanpablomereaotermin/.cache/huggingface/transformers/tmpitcxsidq


Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

storing https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/tokenizer_config.json in cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/b515a7f7f7c5ab818cb49fe535d9886b05f726253b73e8233fba3b1eb1d9dda9.6610639a3a01ef41e1083c8f282f39d442a64ca8691291be4444213bf598269d
creating metadata file for /Users/juanpablomereaotermin/.cache/huggingface/transformers/b515a7f7f7c5ab818cb49fe535d9886b05f726253b73e8233fba3b1eb1d9dda9.6610639a3a01ef41e1083c8f282f39d442a64ca8691291be4444213bf598269d
https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/config.json not found in cache or force_download set to True, downloading to /Users/juanpablomereaotermin/.cache/huggingface/transformers/tmp24j5xqr7


Downloading:   0%|          | 0.00/950 [00:00<?, ?B/s]

storing https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/config.json in cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/b75b62ad64772a1df4c46943b8729e726a8bcc147e845197d591ebde2d1430b2.516dbdb8064a2498055e9af6e5d92ae5a4583bcefe0ca71535c71f42cf513138
creating metadata file for /Users/juanpablomereaotermin/.cache/huggingface/transformers/b75b62ad64772a1df4c46943b8729e726a8bcc147e845197d591ebde2d1430b2.516dbdb8064a2498055e9af6e5d92ae5a4583bcefe0ca71535c71f42cf513138
loading configuration file https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/config.json from cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/b75b62ad64772a1df4c46943b8729e726a8bcc147e845197d591ebde2d1430b2.516dbdb8064a2498055e9af6e5d92ae5a4583bcefe0ca71535c71f42cf513138
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dr

...

storing https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/added_tokens.json in cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/17175619af4cf779d700ee7d96d939b4a2e370552f3e212b28706988c07bc757.3bdabf8f0209d2860f770add80e3711ab238cbb392a68cc562df9a4f94237507
creating metadata file for /Users/juanpablomereaotermin/.cache/huggingface/transformers/17175619af4cf779d700ee7d96d939b4a2e370552f3e212b28706988c07bc757.3bdabf8f0209d2860f770add80e3711ab238cbb392a68cc562df9a4f94237507
https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /Users/juanpablomereaotermin/.cache/huggingface/transformers/tmpx3s0c_ml


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/special_tokens_map.json in cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/1b11459e7e69246d319a1fdec54171a4b3b1420078955aa63daebe50a0639ac5.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /Users/juanpablomereaotermin/.cache/huggingface/transformers/1b11459e7e69246d319a1fdec54171a4b3b1420078955aa63daebe50a0639ac5.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/vocab.txt from cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/182e659b57f089303d9beba9b54d4bbc33e388ea869b153a83344146934df088.6587bde86239957281af55b2f7e564df111a2b4f9dfc0ad884f13ea7106e4dfb
loading file https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/tokenizer.json from cache at /Users/juanpablomereaotermin/.cache/huggingface/transforme

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

storing https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/pytorch_model.bin in cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/3340fd56aeb259af2588d2310cd2b65003da73c71a0b3a9b194261e0f6f7a947.3ee7c714ba16bbcb5520130b897e21deeef4b8547c73faea6878ea623d25bc6a
creating metadata file for /Users/juanpablomereaotermin/.cache/huggingface/transformers/3340fd56aeb259af2588d2310cd2b65003da73c71a0b3a9b194261e0f6f7a947.3ee7c714ba16bbcb5520130b897e21deeef4b8547c73faea6878ea623d25bc6a
loading weights file https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/pytorch_model.bin from cache at /Users/juanpablomereaotermin/.cache/huggingface/transformers/3340fd56aeb259af2588d2310cd2b65003da73c71a0b3a9b194261e0f6f7a947.3ee7c714ba16bbcb5520130b897e21deeef4b8547c73faea6878ea623d25bc6a
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the